In [231]:
import os

In [232]:
!pip install EbookLib beautifulsoup4 lxml
from ebooklib import epub
from bs4 import BeautifulSoup

In [233]:
!pip install mira-sdk

In [234]:
! pip install ElevenLabs

In [235]:
from mira_sdk import MiraClient
client = MiraClient(config={"API_KEY": "YOUR_API_KEY"})


In [236]:
!pip install python-dotenv


In [237]:
os.environ["MIRA_API_KEY"] = "sb-6204f6e14773e5d38a5e26a8bc1f0731"

In [238]:
from dotenv import load_dotenv
from mira_sdk import MiraClient , Flow
import os

load_dotenv()
client = MiraClient(config={"API_KEY": os.getenv("MIRA_API_KEY")})


In [239]:


def extract_chapter_from_epub(epub_path, chapter_number):
    # Load the EPUB file
    book = epub.read_epub(epub_path)

    # Find the items that are HTML files (chapters)
    chapter_list = []
    for item in book.get_items():
        if isinstance(item, epub.EpubHtml):  # Check if the item is an EpubHtml object
            chapter_list.append(item)

    # Ensure the chapter_number is within the valid range
    if chapter_number < 1 or chapter_number > len(chapter_list):
        return "Invalid chapter number."

    # Get the content of the specific chapter
    chapter = chapter_list[chapter_number - 1]  # EPUB chapters are 0-indexed, so adjust
    soup = BeautifulSoup(chapter.content, 'lxml')  # Correct way to access content

    # Extract the text (You can modify this to suit how the text is structured in the EPUB)
    chapter_text = soup.get_text()

    return chapter_text

# Input the path to the EPUB and the chapter number
epub_path = input("Enter the path to the EPUB file: ")  # Provide '/content/SICN.epub' directly here
chapter_number = int(input("Enter the chapter number: "))

# Extract the chapter text
a1 = extract_chapter_from_epub(epub_path, chapter_number)

# Print the extracted text
print(a1)


Enter the path to the EPUB file: /content/SICN.epub
Enter the chapter number: 11
Chapter 10: Heading to the banquetChapter 10: Heading to the banquet'I must say, I look dashing.' Wei Jun commented as he examined himself in the mirror while dressed in a tuxedo. It was a normal tuxedo with nothing special except for the studs yet it looked better on Wei Jun than it did on anyone else.
"Let's go." Wei Jun came out of his room and ordered Chen Xian who had been waiting outside Wei Jun's room for a while now. It was six in the evening and the banquet would start at eight. The son of the birthday woman was already very late.
The car left the villa of Young Ruan and after a drive of twenty minutes, it entered the premises of a mansion five times the size of Young Ruan's villa.
'It's adequate at best in size.' But for a person like Wei Jun or the real one possessing the body, the mansion in front of him wasn't very big.
The car soon stopped in front of a main door. A girl with brown hair and b

<ipython-input-239-5ff9d0fdf185>:17: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(chapter.content, 'lxml')  # Correct way to access content


In [240]:

Prompt_dialogue = str("I have uploaded a chapter of a novel in text format. In this chapter, several characters, including a narrator, speak at different points. Your task is to process the chapter and return the output strictly as a Python list in the format: dialogue_list = [\"Character Name: Their speech\", \"Character Name: Their speech\", ...]. The output must include every line of dialogue or narration in the exact order it appears in the chapter. For each line, identify the speaker (e.g., 'Narrator', 'Alice', 'Bob', etc.) and their corresponding speech. Ensure that the dialogue or narration remains unchanged and strictly follows the specified list format. For example, if the chapter contains the narrator introducing the scene, followed by Alice speaking, Bob responding, and Alice concluding, the output should be: dialogue_list = [\"Narrator: The narrator introduces the scene.\", \"Alice: Hello there!\", \"Bob: How are you?\", \"Alice: I'm fine, thank you!\"]. Process the entire chapter into this format without altering any dialogue or narration content, and ensure the response returned is a valid Python list.")

In [263]:
flow = Flow(source="module-config.yaml")
flow1 = Flow(source="voice-id.yaml")

In [264]:
def dialogue_list(a1, Prompt_dialogue):
    """
    Process a novel chapter with a dialogue extraction prompt and return a single list of dialogues.

    Args:
        a1 (str): The text of the chapter.
        Prompt_dialogue (str): The prompt used to extract dialogue.

    Returns:
        list: A single list of dialogues in the format ["Character Name: Their speech", ...].
    """
    input_dict = {"Chapter": a1, "Prompt": Prompt_dialogue}
    response = client.flow.test(flow1, input_dict)

    # Extract the 'result' key from the API response
    raw_result = response.get('result', '')

    # Debugging: Print the raw result for inspection
    print("Raw result:", raw_result)

    if isinstance(raw_result, str):
        try:
            # Clean up any formatting markers like ```python or ```
            if raw_result.startswith("```python") or raw_result.startswith("```"):
                raw_result = raw_result.strip("```python").strip("```").strip()

            # Extract only the list part by locating the first and last square brackets
            start_index = raw_result.find("[")
            end_index = raw_result.rfind("]") + 1
            if start_index != -1 and end_index != -1:
                list_content = raw_result[start_index:end_index]
            else:
                raise ValueError("Could not locate valid list in the response.")

            # Parse the list content safely
            import ast
            result_array = ast.literal_eval(list_content)  # Converts the string to a Python list

            # Ensure the result is a flat list of strings
            if isinstance(result_array, list) and all(isinstance(item, str) for item in result_array):
                return result_array
            else:
                print("Unexpected format: Parsed result is not a list of strings.")
                return []
        except (SyntaxError, ValueError) as e:
            print(f"Error parsing the response: {e}")
            return []
    else:
        print("Invalid response format: Expected a string.")
        return []


In [265]:
import time

def fetch_dialogue_list():
    while True:  # Infinite loop to restart the process on failure
        try:
            # Assuming dialogue_list is a function that takes a prompt and fetches data from an API
            dialogue_list_result = dialogue_list(a1, Prompt_dialogue)
            return dialogue_list_result  # Return the successful result
        except Exception as e:
            print(f"Error occurred: {e}")
            print("Restarting process from scratch...")
            # Optional: Add a delay before retrying
            time.sleep(5)  # Wait 5 seconds before retrying

# Call the function
a = fetch_dialogue_list()


Raw result: Here is the chapter processed into a Python list of dialogue and narration, with speakers identified:

dialogue_list = [
    "Narrator: 'I must say, I look dashing.' Wei Jun commented as he examined himself in the mirror while dressed in a tuxedo. It was a normal tuxedo with nothing special except for the studs yet it looked better on Wei Jun than it did on anyone else.",
    "Wei Jun: Let's go.",
    "Narrator: Wei Jun came out of his room and ordered Chen Xian who had been waiting outside Wei Jun's room for a while now. It was six in the evening and the banquet would start at eight. The son of the birthday woman was already very late.",
    "Narrator: The car left the villa of Young Ruan and after a drive of twenty minutes, it entered the premises of a mansion five times the size of Young Ruan's villa.",
    "Narrator: 'It's adequate at best in size.' But for a person like Wei Jun or the real one possessing the body, the mansion in front of him wasn't very big.",
    "Nar

In [266]:

type(a)


list

In [267]:
def text_ext(dialogue_list, index):
    """
    Extracts the text after the colon at a specific index in a list.

    Args:
        dialogue_list (list): A list of strings where each string contains 'speaker: dialogue'.
        index (int): The index of the string to process.

    Returns:
        str: The text after the colon in the string at the given index.
    """
    if index < 0 or index >= len(dialogue_list):
        raise IndexError("Index out of range. Please provide a valid index.")

    if ":" not in dialogue_list[index]:
        raise ValueError("Invalid format. The string must contain a ':' to separate the speaker and dialogue.")

    # Extract text after the colon
    text_after_colon = dialogue_list[index].split(":", 1)[1].strip()
    text_before_colon = dialogue_list[index].split(":", 1)[0].strip()

    return text_before_colon,text_after_colon

# Example us
# Extract text after the colon at index 5
result = text_ext(a, 4)
print(result)  # Output: 'So? What have you observed?'


('Narrator', "'It's adequate at best in size.' But for a person like Wei Jun or the real one possessing the body, the mansion in front of him wasn't very big.")


In [268]:
import requests
import json

# Insert your ElevenLabs API key here
API_KEY = "sk_37fbfd685658afa2f68d613cfb96c65f6bda89868a199f4d"

def list_voices():
    url = "https://api.elevenlabs.io/v1/voices"
    headers = {
        "Accept": "application/json",
        "xi-api-key": API_KEY,
        "Content-Type": "application/json"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        voices = response.json().get('voices', [])
        for voice in voices:
            name = voice.get('name', 'N/A')
            description = voice.get('description', 'No description available')
            voice_id = voice.get('voice_id', 'N/A')
            labels = voice.get('labels', {})
            print(f"Name: {name}")
            print(f"Description: {description}")
            print(f"Voice ID: {voice_id}")
            print(f"Labels: {labels}")
            print("-" * 40)
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Example usage
c=list_voices()

Name: Aria
Description: None
Voice ID: 9BWtsMINqrJLrRacOk9x
Labels: {'accent': 'American', 'description': 'expressive', 'age': 'middle-aged', 'gender': 'female', 'use_case': 'social media'}
----------------------------------------
Name: Roger
Description: None
Voice ID: CwhRBWXzGAHq8TQ4Fs17
Labels: {'accent': 'American', 'description': 'confident', 'age': 'middle-aged', 'gender': 'male', 'use_case': 'social media'}
----------------------------------------
Name: Sarah
Description: None
Voice ID: EXAVITQu4vr4xnSDxMaL
Labels: {'accent': 'american', 'description': 'soft', 'age': 'young', 'gender': 'female', 'use_case': 'news'}
----------------------------------------
Name: Laura
Description: None
Voice ID: FGY2WhTYpPnrIDTdsKH5
Labels: {'accent': 'American', 'description': 'upbeat', 'age': 'young', 'gender': 'female', 'use_case': 'social media'}
----------------------------------------
Name: Charlie
Description: None
Voice ID: IKne3meq5aSn9XLyUdCD
Labels: {'accent': 'Australian', 'descripti

In [269]:
prompt_character=str(""" I am giving you voice id below , read the novel and on the basis of traits  of character in the novel  assign character of novel with specific voice id in  the array format, listen dont give the justification only send the array of formatlike [("narrator","voiceId"),("char1","voiceid"),("char2","voiceid1")...] behind this nothing should be returned[
  {"Name": "Aria", "Traits": {"accent": "American", "description": "expressive", "age": "middle-aged", "gender": "female", "use_case": "social media"}},
  {"Name": "Roger", "Traits": {"accent": "American", "description": "confident", "age": "middle-aged", "gender": "male", "use_case": "social media"}},
  {"Name": "Sarah", "Traits": {"accent": "American", "description": "soft", "age": "young", "gender": "female", "use_case": "news"}},
  {"Name": "Charlie", "Traits": {"accent": "Australian", "description": "natural", "age": "middle-aged", "gender": "male", "use_case": "conversational"}},
  {"Name": "George", "Traits": {"accent": "British", "description": "warm", "age": "middle-aged", "gender": "male", "use_case": "narration"}},
  {"Name": "Callum", "Traits": {"accent": "Transatlantic", "description": "intense", "age": "middle-aged", "gender": "male", "use_case": "characters"}},
  {"Name": "River", "Traits": {"accent": "American", "description": "confident", "age": "middle-aged", "gender": "non-binary", "use_case": "social media"}},
  {"Name": "Liam", "Traits": {"accent": "American", "description": "articulate", "age": "young", "gender": "male", "use_case": "narration"}},
  {"Name": "Charlotte", "Traits": {"accent": "Swedish", "description": "seductive", "age": "young", "gender": "female", "use_case": "characters"}},
  {"Name": "Alice", "Traits": {"accent": "British", "description": "confident", "age": "middle-aged", "gender": "female", "use_case": "news"}},
  {"Name": "Matilda", "Traits": {"accent": "American", "description": "friendly", "age": "middle-aged", "gender": "female", "use_case": "narration"}},
  {"Name": "Will", "Traits": {"accent": "American", "description": "friendly", "age": "young", "gender": "male", "use_case": "social media"}},
  {"Name": "Jessica", "Traits": {"accent": "American", "description": "expressive", "age": "young", "gender": "female", "use_case": "conversational"}},
  {"Name": "Eric", "Traits": {"accent": "American", "description": "friendly", "age": "middle-aged", "gender": "male", "use_case": "conversational"}},
  {"Name": "Chris", "Traits": {"accent": "American", "description": "casual", "age": "middle-aged", "gender": "male", "use_case": "conversational"}},
  {"Name": "Brian", "Traits": {"accent": "American", "description": "deep", "age": "middle-aged", "gender": "male", "use_case": "narration"}},
  {"Name": "Daniel", "Traits": {"accent": "British", "description": "authoritative", "age": "middle-aged", "gender": "male", "use_case": "news"}},
  {"Name": "Lily", "Traits": {"accent": "British", "description": "warm", "age": "middle-aged", "gender": "female", "use_case": "narration"}},
  {"Name": "Bill", "Traits": {"accent": "American", "description": "trustworthy", "age": "old", "gender": "male", "use_case": "narration"}}
]""")

In [270]:
def voiceid_list(a1, prompt_character):
    input_dict = {"Chapter": a1, "Prompt": prompt_character}
    response1 = client.flow.test(flow1, input_dict)

    # Extract the 'result' value and convert it to a Python list
    result = eval(response1.get('result', '[]'))  # Using eval to convert the string to a list
    return result



In [271]:

import time

def fetch_character_list():
    while True:  # Infinite loop to restart the process on failure
        try:
            # Assuming voiceid_list is a function that takes a prompt and fetches data from an API
            char_name_list = voiceid_list(a1, prompt_character)
            return char_name_list  # Return the successful result
        except Exception as e:
            print(f"Error occurred: {e}")
            print("Restarting process from scratch...")
            # Optional: Add a delay before retrying
            time.sleep(5)  # Wait 5 seconds before retrying

# Call the function
char_name_list = fetch_character_list()


In [272]:
char_name_list

[('Narrator', 'George'),
 ('Wei Jun', 'Callum'),
 ('Lin Ruan', 'Charlotte'),
 ('Mother', 'Aria'),
 ('Chen Xian', 'Charlie')]

In [273]:
voice_dict = {
    "Aria": "9BWtsMINqrJLrRacOk9x",
    "Roger": "CwhRBWXzGAHq8TQ4Fs17",
    "Sarah": "EXAVITQu4vr4xnSDxMaL",
    "Laura": "FGY2WhTYpPnrIDTdsKH5",
    "Charlie": "IKne3meq5aSn9XLyUdCD",
    "George": "JBFqnCBsd6RMkjVDRZzb",
    "Callum": "N2lVS1w4EtoT3dr4eOWO",
    "River": "SAz9YHcvj6GT2YYXdXww",
    "Liam": "TX3LPaxmHKxFdv7VOQHJ",
    "Charlotte": "XB0fDUnXU5powFXDhCwa",
    "Alice": "Xb7hH8MSUJpSbSDYk0k2",
    "Matilda": "XrExE9yKIg1WjnnlVkGX",
    "Will": "bIHbv24MWmeRgasZH58o",
    "Jessica": "cgSgspJ2msm6clMCkdW9",
    "Eric": "cjVigY5qzO86Huf0OWal",
    "Chris": "iP95p4xoKVk53GoZ742B",
    "Brian": "nPczCjzI2devNBz1zQrb",
    "Daniel": "onwK4e9ZLuTAKqWW03F9",
    "Lily": "pFZP5JQG7iQjIQuC4Bku",
    "Bill": "pqHfZKP75CvOlQylNhV4"
}


In [274]:
def create_character_voice_list(char_name_list, voice_dict):
    # Create a list with (character, name, voiceId)
    result = [
        (character, name, voice_dict.get(name, "VoiceId not found"))
        for character, name in char_name_list
    ]
    return result

# Create the combined list
character_voice_list = create_character_voice_list(char_name_list, voice_dict)

# Output the result
print(character_voice_list)


[('Narrator', 'George', 'JBFqnCBsd6RMkjVDRZzb'), ('Wei Jun', 'Callum', 'N2lVS1w4EtoT3dr4eOWO'), ('Lin Ruan', 'Charlotte', 'XB0fDUnXU5powFXDhCwa'), ('Mother', 'Aria', '9BWtsMINqrJLrRacOk9x'), ('Chen Xian', 'Charlie', 'IKne3meq5aSn9XLyUdCD')]


In [275]:
print(a)

["Narrator: 'I must say, I look dashing.' Wei Jun commented as he examined himself in the mirror while dressed in a tuxedo. It was a normal tuxedo with nothing special except for the studs yet it looked better on Wei Jun than it did on anyone else.", "Wei Jun: Let's go.", "Narrator: Wei Jun came out of his room and ordered Chen Xian who had been waiting outside Wei Jun's room for a while now. It was six in the evening and the banquet would start at eight. The son of the birthday woman was already very late.", "Narrator: The car left the villa of Young Ruan and after a drive of twenty minutes, it entered the premises of a mansion five times the size of Young Ruan's villa.", "Narrator: 'It's adequate at best in size.' But for a person like Wei Jun or the real one possessing the body, the mansion in front of him wasn't very big.", 'Narrator: The car soon stopped in front of a main door. A girl with brown hair and brown eyes was standing at the main door wearing a displeased look. That gir

In [276]:
text_voice_pairs = []

for item in a:
    # Split each entry into character and dialogue
    if ':' in item:
        char, dialogue = item.split(':', 1)
        char = char.strip()  # Remove any extra whitespace
        dialogue = dialogue.strip()  # Remove any extra whitespace

        # Find the voice ID for the character from the character_voice_list
        voice_id = next((voice for c, _, voice in character_voice_list if c == char), None)

        if voice_id:  # Ensure the voice ID exists for the character
            text_voice_pairs.append((dialogue, voice_id))

# The result is already a Python list
print(text_voice_pairs)


[("'I must say, I look dashing.' Wei Jun commented as he examined himself in the mirror while dressed in a tuxedo. It was a normal tuxedo with nothing special except for the studs yet it looked better on Wei Jun than it did on anyone else.", 'JBFqnCBsd6RMkjVDRZzb'), ("Let's go.", 'N2lVS1w4EtoT3dr4eOWO'), ("Wei Jun came out of his room and ordered Chen Xian who had been waiting outside Wei Jun's room for a while now. It was six in the evening and the banquet would start at eight. The son of the birthday woman was already very late.", 'JBFqnCBsd6RMkjVDRZzb'), ("The car left the villa of Young Ruan and after a drive of twenty minutes, it entered the premises of a mansion five times the size of Young Ruan's villa.", 'JBFqnCBsd6RMkjVDRZzb'), ("'It's adequate at best in size.' But for a person like Wei Jun or the real one possessing the body, the mansion in front of him wasn't very big.", 'JBFqnCBsd6RMkjVDRZzb'), ('The car soon stopped in front of a main door. A girl with brown hair and brow

In [277]:
! ELEVENLABS_API_KEY=sk_a2e879e93761e5f4f8b3959ecbbbeab2e8c30a7442fa086b


In [278]:
%env xi-api-key=ELEVENLABS_API_KEY

env: xi-api-key=ELEVENLABS_API_KEY


In [279]:
!pip install pydub

In [280]:
import requests
import json
import os
from pydub import AudioSegment

# Insert your 11Labs API key here
API_KEY = "sk_a2e879e93761e5f4f8b3959ecbbbeab2e8c30a7442fa086b"

def text_to_speech(text, voice_id, output_path, index):
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"

    headers = {
        "Content-Type": "application/json",
        "xi-api-key": API_KEY
    }

    data = {
        "text": text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.8
        }
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        os.makedirs(output_path, exist_ok=True)
        file_name = os.path.join(output_path, f"audio_{index + 1}.mp3")
        with open(file_name, "wb") as audio_file:
            audio_file.write(response.content)
        print(f"Audio saved as {file_name}")
    else:
        print(f"Error: {response.status_code}, {response.text}")

def generate_audio(text_voice_pairs, output_path):
    for index, (text, voice_id) in enumerate(text_voice_pairs):
        text_to_speech(text, voice_id, output_path, index)

def merge_audio_files(output_path, merged_file_path, delay_ms=1000):
    audio_files = sorted([f for f in os.listdir(output_path) if f.endswith(".mp3")])
    combined = AudioSegment.empty()
    previous_voice_id = None

    for index, file in enumerate(audio_files):
        audio = AudioSegment.from_mp3(os.path.join(output_path, file))
        combined += audio

        # Add delay between different audio files
        if index < len(audio_files) - 1:
            combined += AudioSegment.silent(duration=delay_ms)

    combined.export(merged_file_path, format="mp3")
    print(f"Merged audio saved as {merged_file_path}")

# Example usage

output_directory = "/content/audio_FILES"
merged_audio_path = "/content/final_voice.mp3"

# Generate audio sequentially
generate_audio(text_voice_pairs, output_directory)

# Merge audio files



Audio saved as /content/audio_FILES/audio_1.mp3
Audio saved as /content/audio_FILES/audio_2.mp3
Audio saved as /content/audio_FILES/audio_3.mp3
Audio saved as /content/audio_FILES/audio_4.mp3
Audio saved as /content/audio_FILES/audio_5.mp3
Audio saved as /content/audio_FILES/audio_6.mp3
Audio saved as /content/audio_FILES/audio_7.mp3
Audio saved as /content/audio_FILES/audio_8.mp3
Audio saved as /content/audio_FILES/audio_9.mp3
Audio saved as /content/audio_FILES/audio_10.mp3
Audio saved as /content/audio_FILES/audio_11.mp3
Audio saved as /content/audio_FILES/audio_12.mp3
Audio saved as /content/audio_FILES/audio_13.mp3
Audio saved as /content/audio_FILES/audio_14.mp3
Audio saved as /content/audio_FILES/audio_15.mp3
Audio saved as /content/audio_FILES/audio_16.mp3
Audio saved as /content/audio_FILES/audio_17.mp3
Audio saved as /content/audio_FILES/audio_18.mp3
Audio saved as /content/audio_FILES/audio_19.mp3
Audio saved as /content/audio_FILES/audio_20.mp3
Audio saved as /content/audio

In [281]:
output_directory = "/content/audio_FILES"
merged_audio_path = "/content/final_voice/audio.mp3"

In [282]:
merge_audio_files(output_directory, merged_audio_path,delay_ms=500)

Merged audio saved as /content/final_voice/audio.mp3
